In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.sparse
import sys, os

In [2]:

file_path = '/media/thijs/hooghoudt/Zebrafish_data/spontaneous_data_guillaume/'
list_files = [x for x in os.listdir(file_path) if x[-3:] == '.h5']
list_files

['20180911_Run01_rbm1.h5',
 '20180706_Run04_spontaneous_rbm0.h5',
 '20180912_Run01_spontaneous_rbm2.h5',
 '20180913_Run01_rbm3.h5']

In [3]:
i_list = 3
h5view = h5py.File(file_path + list_files[i_list], 'r')

In [4]:
def get_datasets(data_path, obj):
    if isinstance(obj, h5py.Dataset):  # obj is a dataset
        tmp_name = data_path.split('/')[-1].lower()
        all_keys[tmp_name] = obj.name

all_keys = dict()
h5view.visititems(get_datasets)

In [5]:
all_keys

{'baseline': '/Data/Brain/Analysis/Baseline',
 'dff': '/Data/Brain/Analysis/DFF',
 'drifts': '/Data/Brain/Analysis/Drifts',
 'inferredspikes': '/Data/Brain/Analysis/InferredSpikes',
 'thresholdedspikes': '/Data/Brain/Analysis/ThresholdedSpikes',
 'coordinates': '/Data/Brain/Coordinates',
 'ljpcoordinates': '/Data/Brain/LJPCoordinates',
 'labels': '/Data/Brain/Labels',
 'segmentation': '/Data/Brain/Pixels/Segmentation',
 'temporalmean': '/Data/Brain/Pixels/TemporalMean',
 'rawsignal': '/Data/Brain/RawSignal',
 'timedelays': '/Data/Brain/TimeDelays',
 'times': '/Data/Brain/Times',
 'zbraincoordinates': '/Data/Brain/ZBrainCoordinates',
 'trace': '/Data/Stimulus/Spontaneous/Trace',
 'metadata': '/Metadata'}

In [6]:
key = 'thresholdedspikes'
h5view[all_keys[key]]

<HDF5 dataset "ThresholdedSpikes": shape (5000, 44456), type "|u1">

In [7]:
# zebrafish coordinates coordinates to use (use zbrain space)
labels = h5view[all_keys['labels']][:, :].T
inds_wb = np.unique(scipy.sparse.find(labels[:, np.arange(294)])[0])
print(f'# (zbrain) wb neurons: {len(inds_wb)}')
print(f'# total neurons: {h5view[all_keys["zbraincoordinates"]].shape[1]}')
# coordinates = rec.zbrain_coords[inds_wb, :]

# (zbrain) wb neurons: 43439
# total neurons: 44456


In [8]:
spikes_data = h5view[all_keys['thresholdedspikes']][:, :]
assert spikes_data.shape[1] == h5view[all_keys["zbraincoordinates"]].shape[1]
spikes_data = spikes_data[:, inds_wb]
spikes_data.shape

(5000, 43439)

In [9]:
spike_file_name = list_files[i_list][:-3] + '__wb_spikes_only.npy'
np.save(arr=spikes_data, allow_pickle=True,
        file=file_path + spike_file_name)
h5view.close()

In [16]:
# tmp = np.load(file_path + spike_file_name)
# tmp.shape

In [10]:
tmp = None
spikes_data = None